## Building a Burger-vs-Pizza Image Classifier! 

### Task
Given a dataset of color images (consisting of Red, Green, Blue channels) of different food pictures, we want to build an automated classifier that can distinguish between the different classes. To keep it simple we provide pictures of different images of burgers and pizzas. The task then is, *to tell burgers from pizzas*.

### Dataset

The complete dataset is provided as the archive data.zip. When uncompressed it conists of several directories. The names of these directories are the labels of the fruit type. Each directory has numerous pictures of the food type in JPG format. These have been scraped from the web with permissive licenses.

In [ ]:
# Some of the essential Python Packages to be loaded
# See how you can load a package using a different name; 
import numpy as np 
import matplotlib.pyplot as plt
import glob
from skimage.transform import resize

# add any other packages you might need

In [ ]:
## Read the folder names with the data
food_names = glob.glob('data/*')
print(food_names)

In [ ]:
burger_list = glob.glob('data/burger/*')
pizza_list = glob.glob('data/pizza/*')
# print(ap_list)

In [ ]:
# The data is provided in two folders with names
# burger and pizza.
# Read the file names using glob.glob()

# Count the number of burger and pizza images
nBurger = len(burger_list)## to be completed
nPizza = len(pizza_list) ## to be completed
print("Dataset contains %d burgers and %d pizza images" %(nBurger,nPizza))

In [ ]:
# Show some sample burger and pizza using pyplot
plt.figure(figsize=(12,8))
plt.gray()
imgA = plt.imread(burger_list[0])
plt.subplot(241)
plt.imshow(imgA[:,:,0],cmap='Reds_r')
plt.axis('off')
plt.subplot(242)
plt.imshow(imgA[:,:,1],cmap='Blues_r')
plt.axis('off')
plt.subplot(243)
plt.imshow(imgA[:,:,2],cmap='Greens_r')
plt.axis('off')
plt.subplot(244)
plt.imshow(imgA[:,:])
plt.axis('off')
imgA.shape

In [ ]:
### Images are matrices. So, one can compute means over different axes!
imgA.mean((0,1))

###
H, W, C = imgA.shape[0], imgA.shape[1], imgA.shape[2]
print(H,W,C)

In [ ]:
# We would like to extract some features 
# to be able to compare burgers vs pizzas

# Can  you think of what features might be 
# most useful?

nFeatures = 3 
# Initialise empty arrays to hold some features
burgerFeatures = np.zeros((nBurger,nFeatures))
pizzaFeatures = np.zeros((nPizza,nFeatures))

In [ ]:
# We need to assign "labels" to distinguish 
# burgers from pizzas

# Usually the two classes are mapped to two numbers
# We use 0 -> burger and 1 -> pizza
labels = ['burger','pizza']
burgerLabels = np.ones(nBurger)*0
pizzaLabels = np.ones(nPizza)*1

In [ ]:
# Feature extraction 
# One of the basic features that can be extracted is the average intensity
# of the R, G, B channels. 
# Steps:
# 1. Load image data
# 2. Convert it into numpy array
# 3. Extract 3 features 

burgerImages = np.zeros((nBurger,H,W,C))
for i in range(nBurger):
    img = plt.imread(burger_list[i])
    burgerImages[i] = img
    # img = resize(img,(64,64))
    burgerFeatures[i] = img.mean((0,1))
    


In [ ]:
pizzaImages = np.zeros((nBurger,H,W,C))
for i in range(nPizza):
    img = plt.imread(pizza_list[i])
    pizzaImages[i] = img
    # img = resize(img,(64,64))
    pizzaFeatures[i] = img.mean((0,1))


In [ ]:
# Visualise the pairwise features
plt.scatter(burgerFeatures[:,0],burgerFeatures[:,2],marker='o')
plt.scatter(pizzaFeatures[:,0],pizzaFeatures[:,2],marker='x')
plt.xlabel('Average-Red')
plt.ylabel('Average-Blue')

In [ ]:
from mpl_toolkits.mplot3d import Axes3D 

In [ ]:
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(111,projection='3d')
ax.scatter(burgerFeatures[:,0],burgerFeatures[:,1],
           burgerFeatures[:,2],c='r',marker='o',label='burger')
ax.scatter(pizzaFeatures[:,0],pizzaFeatures[:,1],
           pizzaFeatures[:,2],c='g',marker='^',label='pizza')
ax.set_xlabel('Average-R')
ax.set_ylabel('Average-G')
ax.set_zlabel('Average-B')
plt.legend()
ax = plt.gca()
ax.xaxis.set_ticklabels([]);
ax.yaxis.set_ticklabels([]);
ax.zaxis.set_ticklabels([]);

In [ ]:
# make the dataset
# Combine features of both burgers and pizzas into
# a single array. Same for labels. 

### If true all image pixels will be used as features!
use_image_as_features = False
channels = [0,1,2]
if use_image_as_features:
    X = np.concatenate((burgerImages.reshape(nBurger,-1),pizzaImages.reshape(nPizza,-1)),axis=0)
    print(X.shape)
else:
    X = np.concatenate((burgerFeatures,pizzaFeatures),axis=0)
    print(X.shape)
Y = np.concatenate((burgerLabels,pizzaLabels))
print(Y.shape)

### Shuffle the dataset
idx = np.random.permutation(len(X))
X = X[idx]
Y = Y[idx]

In [ ]:
# Let us shuffle the data! 
from sklearn.model_selection import train_test_split
xTrain, xTest, yTrain, yTest = train_test_split(X,Y,test_size=0.25)
print(xTrain.shape,xTest.shape)

In [ ]:
# Now that our data is ready. Let us use a classifier.

# We will use another linear classifier
# Logistic regression learns a linear boundary 
# to classify binary class of inputs

from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

### You can try other models too

In [ ]:
clf = LogisticRegression().fit(xTrain, yTrain)
# Print the model coefficients.
# print("Logistic regression parameters: ",clf.coef_[0],clf.intercept_[0])

# clf = Perceptron().fit(xTrain, yTrain)
# clf = RandomForestClassifier().fit(xTrain, yTrain)
# clf = MLPClassifier().fit(xTrain, yTrain)

In [ ]:
# Check the performance on the training dataset
yTrPred = clf.predict(xTrain)
print(clf.score(xTrain,yTrain))
print(clf.score(xTest,yTest))

In [ ]:
### Inference for new data
new_data = (glob.glob('data/for_prediction/*.jpg'))
N = len(new_data)
print('Found %d files for prediction.'%N)

In [ ]:
import pandas as pd
pred_df = pd.DataFrame(columns=['name','pred'],index=np.arange(N))

In [ ]:
nCol = 4
nRow = N//nCol
plt.figure(figsize=(8,32))

for i in range(N):
    
    print('Loading ',new_data[i])
    img = plt.imread(new_data[i])
    if xTrain.shape[1] == 3:
        xPred = img.mean((0,1)).reshape(1,-1)
    elif xTrain.shape[1] == H*W*C:
        xPred = img.reshape(1,-1)
    else:
        print('Incorrect model!!!')
        break
    
#     X_inference.shape
    pred = int(clf.predict(xPred)[0])
    print(labels[pred])
    plt.subplot(nRow,nCol,i+1)
    plt.imshow(img)
    plt.axis('off')
    plt.title('Pred:'+labels[pred])

    ### Write results in dataframe
    pred_df.loc[i,'name'] = new_data[i]
    pred_df.loc[i,'pred'] = labels[pred]

### Save results in csv
pred_df.to_csv('predictions.csv')